## Preprocessing

In [147]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [148]:
# Determine the number of unique values in each column.
# The code provided is a for loop that counts the number of each unique value within a column and then displays it after the variable
for x in application_df.columns:
    print(f"{x} {len(application_df[x].unique())}")

EIN 34299
NAME 19568
APPLICATION_TYPE 17
AFFILIATION 6
CLASSIFICATION 71
USE_CASE 5
ORGANIZATION 4
STATUS 2
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 8747
IS_SUCCESSFUL 2


In [149]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME', 'AFFILIATION', 'CLASSIFICATION'])

In [150]:
# Look at APPLICATION_TYPE value counts for binning
# This line of code is APPLICATION_TYPE contains categorical data and as a result we need to bin it by counting
# the number of instances that each variable contains.
app_type = application_df['APPLICATION_TYPE'].value_counts()
app_type

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [151]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# I used a different comnbination of app_type < number such as 200,300, and so forth until I got to one that
# resembles what was given of having other being equal to 276
# The line of code is also providing data that there must be a threshhold of 500 instances in order to stay grouped. since we chose 500
# All variables that contained less than that got grouped into other
application_types_to_replace = list(app_type[app_type<500].index)
application_types_to_replace

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [152]:
income = application_df['INCOME_AMT'].value_counts()
income

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [153]:
# Choose a cutoff value and create a list of application types to be replaced
income_to_replace = list(income[income<500].index)
income_to_replace

# Replace in dataframe
for app in income_to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(app,"Millions")

# Check to make sure binning was successful
application_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
Millions           564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [154]:
USE = application_df['USE_CASE'].value_counts()

In [155]:
case_use = USE[USE>1]
case_use

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [156]:
# Convert categorical data to numeric with `pd.get_dummies`
# As the comment suggests above we are converting the data to be numerical in order to implement
# machine learning. as you can see from the data table below the pd.get_dummies is converting the values into a 0.0 or 1.0
application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Millions,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [157]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL']).values
y = application_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42)

In [158]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [162]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model - deep neural net
nn = Sequential()

# First hidden layer
nn.add(Dense(units=128, activation='relu', input_dim=29))

# Second hidden layer
nn.add(Dense(units=64, activation='relu'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

nn.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 128)               3840      
                                                                 
 dense_43 (Dense)            (None, 64)                8256      
                                                                 
 dense_44 (Dense)            (None, 1)                 65        
                                                                 
Total params: 12,161
Trainable params: 12,161
Non-trainable params: 0
_________________________________________________________________


In [163]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [165]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.15, epochs=100)

Epoch 1/100
684/684 [==============================] - 1s 2ms/step - loss: 0.6355 - accuracy: 0.6363 - val_loss: 0.6381 - val_accuracy: 0.6344
Epoch 2/100
684/684 [==============================] - 1s 2ms/step - loss: 0.6350 - accuracy: 0.6361 - val_loss: 0.6406 - val_accuracy: 0.6346
Epoch 3/100
684/684 [==============================] - 1s 2ms/step - loss: 0.6350 - accuracy: 0.6374 - val_loss: 0.6383 - val_accuracy: 0.6344
Epoch 4/100
684/684 [==============================] - 1s 1ms/step - loss: 0.6349 - accuracy: 0.6358 - val_loss: 0.6377 - val_accuracy: 0.6325
Epoch 5/100
684/684 [==============================] - 1s 2ms/step - loss: 0.6346 - accuracy: 0.6365 - val_loss: 0.6352 - val_accuracy: 0.6346
Epoch 6/100
684/684 [==============================] - 1s 2ms/step - loss: 0.6343 - accuracy: 0.6376 - val_loss: 0.6364 - val_accuracy: 0.6338
Epoch 7/100
684/684 [==============================] - 1s 1ms/step - loss: 0.6343 - accuracy: 0.6365 - val_loss: 0.6382 - val_accuracy: 0.6331

In [166]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6462 - accuracy: 0.6309 - 249ms/epoch - 928us/step
Loss: 0.6461798548698425, Accuracy: 0.6309037804603577


In [ ]:
# Export our model to HDF5 file
from google.colab import files

nn.save('/content/AlphabetSoupCharity_Optimization.h5')
files.download('/content/AlphabetSoupCharity_Optimization.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>